<div dir="rtl">
از اونجا که خیلی دقیق گفته نشده بود قراره چطوری از این دیتاکالکتور استفاده بشه نسخه‌ای که اینجا هست یکم با گیت‌هاب (وفایل data_getter.py) فرق داره.

ابتدا روش نوشتن یک دیتا کالکتور گفته می‌شود (یعنی نوشتن data_getter.py) و سپس استفاده از data_getter.py است.
</div>

<div dir="rtl">
نصب پیش نیاز‌ها


In [1]:
!pip install tweepy
!pip install pandas

<div dir="rtl">
کلاس های زیر باید به پروژه اضافه شوند.

In [42]:
import time
from typing import List, Callable, Optional, Tuple

import tweepy
import urllib3
import pandas as pd
from tweet import Tweet

<div dir="rtl">
برای ایجاد api در 
<a href="https://developer.twitter.com/en/apply/user"> اینجا </a> اقدام کنید
و یک 
app
جدید بسازسید. بعد از ساختن و اضافه کردن 3-legged OAuth
۲ تا توکن و ۲ تا سکرت توکن به شما داده می‌شود
<div>

In [87]:
"""
consumer_key: API key
consumer_secret: API secret
access_token: 3-legged OAuth token
access_token_secret: 3-legged OAuth secret
"""
consumer_key=""  # API key
consumer_secret=""  # API secret
access_token=""
access_token_secret=""

<div dir="rtl">
با استفاده از پکیج tweepy به api توئیتر وصل می‌شویم
<br>
wait_on_rate_limit برای این است که هنگامی که به لیمیت api توئیتر رسیدیم به صورت خودکار صبر کند و wait_on_rate_limit_notify نیز وقتی به لیمیت رسیدیم با print به کاربر اعلام می‌کند
<div>

In [44]:
# set auth
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# set api
api = tweepy.API(
    auth,
    wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True
)


<div dir="rtl">
برای تایید توکن‌ها از تابع زیر می‌توانید استفاده کنید.

In [45]:
try:
  if api.verify_credentials():
    print('authentication verifed')
  else:
    print('authentication failed')
except:
  print('authentication failed')

authentication verifed


<div dir="rtl">
هرچند خود پکیج tweepy شی Status را برای توئیت‌ها دارد اما برای راحتی مدل خودمان را پیاده سازی می‌کنیم. (status همان tweet است)
</div>

In [46]:
from tweet import Tweet

<div dir="rtl">
api توئیتر لیمیت تعداد توئیتی که می‌دهد ۲۰۰ است برای همین اگر می‌خواهیم بیشتر از آن توئیت بگیریم باید چند بار endpoint ها را صدا کنیم
<br>
تابع زیر متد method را که خروجی آن لیستی از Status است را چند بار صدا می‌زند تا به تعداد مشخص شده توئیت از API بگیرد.
</div>

In [47]:
MAX_COUNT = 200
def collect_tweets(method, count, *args, **kwargs) -> List[tweepy.Status]:
    """
    Tweeter API allow maximum 200 tweets to get
    so if we want to collect more than 200 we have
    to call method more than once

    Args:
        method: method to call
        count: number of tweets to collect
        *args: args of method
        **kwargs: kargs of methd

    Returns:

    """
    cnt = count
    oldest_id = None
    result: List[tweepy.Status] = []
    while cnt:
        if oldest_id is None:
            new_tweets = (
                method(
                    count=min(MAX_COUNT, cnt),
                    *args,
                    **kwargs
                )
            )
        else:
            new_tweets = (
                method(
                    count=min(MAX_COUNT, cnt),
                    max_id=oldest_id - 1,
                    *args,
                    **kwargs
                )
            )
        if len(new_tweets) == 0:
            break
        oldest_id = new_tweets[-1].id
        result.extend(new_tweets)
        cnt -= len(new_tweets)
    return result

<div dir="rtl">
برای گرفتن تایم لاین کاربر (فردی که توکن api برای آن است از تابع api.home_timeline می‌شود استفاده کرد.
<br>
همانطور که گفته شد مدل خودمان را از توئیت پیاده سازی کردیم اما تابع ذکر شده خروجی اش Status است پس باید این شی را به Tweet تبدیل کرد
<br>
دکوریتور parse_tweetsاینکار را انجام می‌دهد.
</div>

In [48]:
def parse_tweets(func):
    """
    Cast tweepy.Status to tweet.Tweet
    """

    def outer_parser(*args, **kwargs) -> List[Tweet]:
        return [Tweet.deserializer(twt._json) for twt in func(*args, **kwargs)]

    return outer_parser
@parse_tweets
def get_timeline_tweets(count: int) -> List[Tweet]:
    """
    Gets timeline tweets of api owner
    Args:
        count: number of tweets to get
    """
    return collect_tweets(
        api.home_timeline,
        count=count,
        tweet_mode='extended',
    )

In [49]:
get_timeline_tweets(5)

[Tweet(created_at='Fri Jul 02 10:15:49 +0000 2021', id=1410905041813319680, full_text='https://t.co/JauFV90aZN', source='<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', in_reply_to_status_id=None, in_reply_to_user_id=None, in_reply_to_screen_name=None, user=TweetUser(id=824696073491017729, name='I told my therapist about you', screen_name='itoldmythe', location='', url='https://t.co/8zFIu1hzwn', description='👀', protected=False, verified=False, followers_count=159600, friends_count=6, listed_count=339, favourites_count=261, statuses_count=4087, created_at='Thu Jan 26 19:10:40 +0000 2017', profile_banner_url='https://pbs.twimg.com/profile_banners/824696073491017729/1613909454', profile_image_url_https='https://pbs.twimg.com/profile_images/1363460782621683714/F9-kqpaB_normal.jpg', default_profile=False, default_profile_image=False, withheld_in_countries=[]), coordinates=None, place=None, is_quote_status=False, quoted_status_id=None, quoted_status=None

<div dir="rtl">
با استفاده از تابع api.user_timeline
می‌شود توئیت‌های کاربرهای مورد نظر را گرفت
</div>

In [50]:
@parse_tweets
def get_historical(count: int, users: List[str]) -> List[Tweet]:
    """
    Gets all tweets of users
    Args:
        count: number of tweets of each user
        users: list of users id
    """
    result = []
    for user_id in users:
        result.extend(
            collect_tweets(
                api.user_timeline,
                count=count,
                screen_name=user_id,
                tweet_mode='extended',
            )
        )
    return result
  

In [51]:
get_historical(50, ['BTCTN', 'teat65501837'])

[Tweet(created_at='Fri Jul 02 10:12:12 +0000 2021', id=1410904130936360963, full_text='Enjin successfully auctioned all tokens available for sale for the new Efinity #NFT marketplace it has developed for the Polkadot ecosystem. https://t.co/IRFpsKLUfS', source='<a href="https://zapier.com/" rel="nofollow">Zapier.com</a>', in_reply_to_status_id=None, in_reply_to_user_id=None, in_reply_to_screen_name=None, user=TweetUser(id=3367334171, name='Bitcoin News', screen_name='BTCTN', location='World Wide', url='https://t.co/oZCuHV2ZwX', description='Official Twitter account for https://t.co/Y3Sm6N0oR5. Follow us for all the latest Bitcoin news and services! Follow @bitcoincom for all https://t.co/5vSQA17h1m updates.', protected=False, verified=True, followers_count=1476740, friends_count=928, listed_count=7520, favourites_count=10091, statuses_count=20323, created_at='Thu Jul 09 08:58:22 +0000 2015', profile_banner_url='https://pbs.twimg.com/profile_banners/3367334171/1560756921', profile_image

<div dir="rtl">
با استفاده از قابلیت stream که پکیج tweepy دارد می‌شود تعدادی کاربر مشخص کرد و دیتای آن‌ها به صورت زنده (با تاخیر چند ثانیه) گرفت.
<br>
برای اینکار می‌توانیم کلاس Listener خودمان را پیاده سازی کنیم.
<br>
این کلاس باید فرزند tweepy.StreamListener باشد
<br>
فعلا فقط تابع on_status که زمانی که توئیت جدیدی زده می‌شود صدا زده می‌شود و تابع on_error پیاده سازی شده اند.
</div>

In [52]:
class Listener(tweepy.StreamListener):
    """
    Custom stream listener to call callback method
    """

    def __init__(self, api, callback: Callable[[Tweet], None]):
        super().__init__(api)
        self.__callback = callback

    def on_status(self, tweet):
        self.__callback(Tweet.deserializer(tweet._json))

    def on_error(self, status_code):
        print('error')
        print(status_code)

def stream(users: List[str], callback: Callable[[Tweet], None]) -> None:
    """
    Get new tweets from users

    Args:
        users: list of users id
        callback: callback method
    """
    users_id = [user._json['id_str'] for user in api.lookup_users(screen_names=users)]
    listener = Listener(api, callback)
    stream = tweepy.Stream(auth=api.auth, listener=listener)
    while True:
        try:
            stream.filter(follow=users_id)
        except  urllib3.exceptions.ProtocolError as e:
            print('error')
            print(e)
            time.sleep(15)

In [54]:
stream(['teat65501837'], lambda tweet: print(tweet))

Tweet(created_at='Fri Jul 02 10:29:16 +0000 2021', id=1410908423131176960, full_text='test2', source='<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', in_reply_to_status_id=None, in_reply_to_user_id=None, in_reply_to_screen_name=None, user=TweetUser(id=1146534916064251904, name='teat', screen_name='teat65501837', location=None, url=None, description='just testing tweeter api :)', protected=False, verified=False, followers_count=0, friends_count=0, listed_count=0, favourites_count=0, statuses_count=10, created_at='Wed Jul 03 21:43:20 +0000 2019', profile_banner_url='https://pbs.twimg.com/profile_banners/1146534916064251904/1625218462', profile_image_url_https='https://pbs.twimg.com/profile_images/1410894471194087429/h58t3EU4_normal.jpg', default_profile=True, default_profile_image=False, withheld_in_countries=[]), coordinates=None, place=None, is_quote_status=False, quoted_status_id=None, quoted_status=None, retweet_count=0, favorite_count=0, entities=Entities(h

KeyboardInterrupt: ignored

<div dir="rtl">
از آنجا که اطلاعات توئیت‌ها خیلی زیاد است تابع زیر کمک می‌کند فقط اطلاعات پایه را چاپ کند.
</div>

In [ ]:
def pretty_print(tweet: Tweet) -> str:
    if tweet.retweeted_status and not tweet.retweeted_status.quoted_status:
        return f'A new retweet\nsender: {tweet.user.screen_name}\nretweet from: {tweet.retweeted_status.user.screen_name}\ntext: {tweet.retweeted_status.full_text}'
    elif tweet.retweeted_status:
        return f'A new retweet\nsender: {tweet.user.screen_name}\nretweet from: {tweet.retweeted_status.user.screen_name}\nquoted from: {tweet.retweeted_status.quoted_status.user.screen_name}\ntext: {tweet.retweeted_status.full_text}\norigin text: {tweet.retweeted_status.quoted_status.full_text}'
    elif tweet.quoted_status:
        return f'A new quoted tweet\nsender: {tweet.user.screen_name}\nquoted from: {tweet.quoted_status.user.screen_name}\nquote text: {tweet.full_text}\norigin text: {tweet.quoted_status.full_text}'
    else:
        return f'A new tweet\nsender: {tweet.user.screen_name}\ntext: {tweet.full_text}'

<div dir="rtl">
تابع زیر لیستی از توئیت‌ها را به دیتافریم تبدیل می‌کند
<br>
از آنجایی که یک توئیت مممکن است ریتوئیت یک retweet with quote باشد تابع get_quote کاربر و متن توئیتی که کوت شده است را بر می‌گرداند.
</div>

In [70]:
def get_quote(tweet: Tweet) -> Tuple[Optional[str], Optional[str]]:
    """
    Returns None, None if tweet is not quote
    otherwise return tuple of quoted tweet sender and text
    """
    if tweet.retweeted_status:
        return get_quote(tweet.retweeted_status)
    if not tweet.quoted_status:
        return None, None
    return tweet.quoted_status.user.screen_name, tweet.quoted_status.full_text


def tweets_to_df(tweets: List[Tweet]) -> pd.DataFrame:
    """
    Cast Tweet to df
    """
    return pd.DataFrame(
        [[
            tweet.created_at,
            tweet.user.screen_name,
            tweet.full_text,
            True if tweet.retweeted_status else False,
            None if not tweet.retweeted_status else tweet.retweeted_status.user.screen_name,
            True if get_quote(tweet)[0] else None,
            get_quote(tweet)[0],
            get_quote(tweet)[1]
        ] for tweet in tweets],
        columns=[
            'create time',
            'sender',
            'text',
            'is_retweet',
            'retweet_sender',
            'is_quote',
            'quoted_tweet_sender',
            'quoted_tweet_text'
        ]
    )

<div dir="rtl">
تابع‌های گفته شده در کلاس DataGetter پیاده سازی شده است.

In [71]:
from data_getter import *

<div dir="rtl">
ایجاد یک دیتا کالکتور:

In [75]:
dg = DataGetter(
    consumer_key="",  # API key
    consumer_secret="",  # API secret
    access_token="",
    access_token_secret="",
)

<div dir = "rtl">
تایید احراز هویت

In [76]:
print(dg.verify_authentication())

User(_api=<tweepy.api.API object at 0x7fef6441bad0>, _json={'id': 1285595741080170501, 'id_str': '1285595741080170501', 'name': 'Mahdi', 'screen_name': 'MahdiShobeiri', 'location': 'Tehran', 'description': '\u200f\u200f\u200f\u200f\u200fسید مهدی صادق شبیری\nخوشبختانه مهندسی کامپیوتر\nمتأسفانه در سوت(سات؟)\nTelegram: @this_is_my_per_id', 'url': 'https://t.co/bmYOdidpiX', 'entities': {'url': {'urls': [{'url': 'https://t.co/bmYOdidpiX', 'expanded_url': 'https://t.me/Music2BeBuriedAliveBy', 'display_url': 't.me/Music2BeBuried…', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 80, 'friends_count': 102, 'listed_count': 0, 'created_at': 'Tue Jul 21 15:21:29 +0000 2020', 'favourites_count': 1979, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 248, 'lang': None, 'status': {'created_at': 'Thu Jul 01 17:51:44 +0000 2021', 'id': 1410657386726137859, 'id_str': '1410657386726137859', 'text': '@lTheMightyMl

<div dir = "rtl">
گرفتن تایم لاین

In [77]:
for tweet in dg.get_timeline_tweets(5):
  print(pretty_print(tweet), end="\n--------\n")

A new retweet
sender: asra19366209
retweet from: Khatesevvom
text: ‌
‌
                از دِهی عبور کردم
                که در آنجا راهبه‌ای بود.
                گفتم: آیا مکانی پاک یافت می‌شود
                که نماز بگزارم؟
                گفت: دلت را پاک کن
                و در هرکجا که خواستی نماز بگزار.

                #بایزید_بسطامی
‌
‌
--------
A new tweet
sender: itoldmythe
text: https://t.co/JauFV90aZN
--------
A new retweet
sender: khazanpm
retweet from: Hozak57
text: کشوری که غنی از نفت و گاز و کوه طلا است 
سهمش برای ما #کولبری بود https://t.co/1P9mKiPW2Q
--------
A new tweet
sender: yaasamn
text: رییس اینا همه‌ش بازی بود. تو گروگان من بودی.
--------
A new tweet
sender: Ashkanj78
text: انقدر که من برای شاگردام بغض میکنم و نگرانشونم و میخوام مثل من تر نزنن به ایندشون، خانواده هاشون درگیر نیستن.
--------


<div dir = "rtl">
گرفتن توئیت‌های کاربرها

In [78]:
for tweet in dg.get_historical(50, ['teat65501837']):
    print(pretty_print(tweet), end="\n---------------\n")


A new tweet
sender: teat65501837
text: test3
---------------
A new tweet
sender: teat65501837
text: test2
---------------
A new tweet
sender: teat65501837
text: test
---------------
A new tweet
sender: teat65501837
text: this tweet has a #hash_tag
also has a url https://t.co/XTZZ5oc1Xy
@MahdiShobeiri some mention too!
---------------
A new retweet
sender: teat65501837
retweet from: teat65501837
quoted from: teat65501837
text: long quote from a long tweet long quote from a long tweet long quote from a long tweet long quote from a long tweet long quote from a long tweet long quote from a long tweet long quote from a long tweet long quote from a long tweet long quote from a long tweet https://t.co/ceSwbyjnPo
origin text: this is a long tweet this is a long tweet this is a long tweet this is a long tweet this is a long tweet this is a long tweet this is a long tweet this is a long tweet this is a long tweet this is a long tweet this is a long tweet this is a long tweet this is a long tweet

<div dir = "rtl">
استریم کردن چند کاربر
(اکسپشن مهم نیست چون پایان دادم بهش اکسپشن داده)

In [79]:
dg.stream(['teat65501837'], lambda tweet: print(pretty_print(tweet), end="\n---------------\n"))

KeyboardInterrupt: ignored

<div dir="rtl">
تبدیل توئیت به دیتا فریم

In [86]:
tweets_to_df(dg.get_historical(50, ['teat65501837']))


,create time,sender,text,is_retweet,retweet_sender,is_quote,quoted_tweet_sender,quoted_tweet_text
0,Fri Jul 02 10:36:07 +0000 2021,teat65501837,test3,False,None,None,None,None
1,Fri Jul 02 10:29:16 +0000 2021,teat65501837,test2,False,None,None,None,None
2,Fri Jul 02 10:25:50 +0000 2021,teat65501837,test,False,None,None,None,None
3,Fri Jul 02 09:18:14 +0000 2021,teat65501837,this tweet has a #hash_tag\nalso has a url htt...,False,None,None,None,None
4,Fri Jul 02 08:51:09 +0000 2021,teat65501837,RT @teat65501837: long quote from a long tweet...,True,teat65501837,True,teat65501837,this is a long tweet this is a long tweet this...
5,Fri Jul 02 08:50:52 +0000 2021,teat65501837,long quote from a long tweet long quote from a...,False,None,True,teat65501837,this is a long tweet this is a long tweet this...
6,Fri Jul 02 08:50:27 +0000 2021,teat65501837,quote from a tweet https://t.co/clGsgVtOZU,False,None,True,teat65501837,this is a short tweet
7,Fri Jul 02 08:49:59 +0000 2021,teat65501837,RT @Apple: Von Dresden über Rüspel bis Berlin ...,True,Apple,None,None,None
8,Fri Jul 02 08:49:48 +0000 2021,teat65501837,RT @teat65501837: this is a short tweet,True,teat65501837,None,None,None
9,Fri Jul 02 08:49:39 +0000 2021,teat65501837,this is a long tweet this is a long tweet this...,False,None,None,None,None
